In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
import matplotlib.colors
from matplotlib.gridspec import GridSpec
from mpl_toolkits.mplot3d import Axes3D

# Import colormaps and limits
import sys
sys.path.append('./')
from colormaps import *

In [ ]:
# initial geometry
data_dir = '../../Zenodo/model_output'

la_M_IM = xr.open_dataset(f'{data_dir}/LA_M.nc')

ini_geom = la_M_IM.sel(time=0)

In [ ]:
# Additional colorbars

# bathymetry/bed
cmap_bed = 'BrBG'
norm_bed = mpl.colors.Normalize(vmin=-750,vmax=750)

# surface
cmap_surf ='Greys_r'
norm_surf =  mpl.colors.Normalize(vmin=0, vmax=2000)


In [ ]:
# Plot 3D view of initial geometry
ds = ini_geom

# Extract variables
Hb = ds['Hb']       # Bed geometry
Hs = ds['Hs']       # Surface thickness
Hi = ds['Hi']       # Ice thickness
mask = ds['mask']   # Mask

# Define the coordinates
x = Hb.coords['x']  # Assuming 'x' is the horizontal coordinate
y = Hb.coords['y']  # Assuming 'y' is the horizontal coordinate

# Calculate the bed, surface elevation and draft elevation, remove ocean from data
Z_bed           = Hb                                                                                   # Bed elevation
Z_surface_gr    = xr.where(Hs > 0, Hs, np.nan)                                                         # Surface elevation ice : if you want to select only grounded, add: * xr.where(np.logical_or(mask==3, mask==7), 1, np.nan)
Z_surface_fl    = xr.where(mask>3, 1, np.nan)* xr.where(Hs > 0, Hs, np.nan)                            # Surface elevation floating ice : if you want to plot ice shelf in different colormap
Z_draft         = xr.where(Hs-Hi>Hb+.1,1,np.nan) * xr.where(Hs > 0, Hs - Hi, np.nan)                   # Draft elevation

# Create a 3D plot
fig = plt.figure(figsize=(10,8),dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Convert coordinates to numpy arrays
x = x.values
y = y.values

# Create a meshgrid for the x, y coordinates
X, Y = np.meshgrid(x / 1000, y / 1000)

# Plot bed geometry (Bed should be plotted first to ensure surface is on top)
im_bed      = ax.plot_surface(X, Y, Z_bed,          cmap=cmap_bed,      norm=norm_bed,      alpha=1, zorder=1, label='Bed Geometry')

# Plot surface elevation (Make sure it is plotted after the bed to appear in front)
im_surfgr   = ax.plot_surface(X, Y, Z_surface_gr,   cmap=cmap_surf,     norm=norm_surf,     alpha=1, zorder=2, label='Surface Elevation')

# If you want to plot ice shelf in separate colormap:
#ax.plot_surface(X, Y, Z_surface_fl, cmap='Greys_r',norm=mpl.colors.Normalize(vmin=600, vmax=700), alpha=1, zorder=2, label='Surface Elevation - floating')

# Optional: Plot draft elevation if needed
im_draft    = ax.plot_surface(X, Y, Z_draft,        cmap=cmap_draft,    norm=norm_draft,    alpha=1, zorder=0, label='Draft Elevation')

# Add labels and title
ax.set_xlabel('X [km]',labelpad=20)
ax.set_ylabel('Y [km]',labelpad=1)
ax.set_zlabel('Elevation [m]',labelpad=1)

# Set limits for better visualization
ax.set_xlim([-400, 400])
ax.set_ylim([-40, 40])

# Set viewpoint and aspect ratio for better viewing angles
ax.view_init(elev=40, azim=-70)
ax.set_box_aspect([1, 0.1, 0.5])  # Ratio of x:y:z

# Sex ticks and ticklabels
ax.set_xticks([-400,-300,-200,-100,0,50,100,200,240,300,400])
ax.set_xticklabels([-400,-300,-200,-100,0,'',100,200,'',300,400])
ax.set_yticks([-40,0,40])
ax.set_yticklabels([-40,0,40],rotation=-5,
                   verticalalignment='baseline',
                   horizontalalignment='left')

# Show and save plot
fig.savefig('fig02_panel_a.png', dpi=300, bbox_inches='tight')


In [ ]:
# Plot colorbars
fig,ax = plt.subplots(1,6,figsize=(3.5,2),width_ratios=[.5,2.5,.5,2.5,.5,2.5],dpi=300)
plt.colorbar(im_surfgr, ax[0],orientation='vertical',extend='max',  label='Surface height [m]')
plt.colorbar(im_draft, ax[2],orientation='vertical', extend='min', label='Ice draft depth [m]')
plt.colorbar(im_bed, ax[4],orientation='vertical', extend='both', label='Bathymetry [m]')

for i in [1,3,5]:
    ax[i].set_axis_off()

fig.savefig('fig02_colorbars.png', dpi=300)

In [ ]:
# Plot topview bathymetry and ice geometry

fig = plt.figure(figsize=(3.5,20), dpi=300)
gs = GridSpec(3,3, width_ratios=[1,0.1,1], hspace=0.1, wspace=0.2)

# Create axes
ax0 = fig.add_subplot(gs[0])                            
ax1 = fig.add_subplot(gs[2], sharex=ax0, sharey=ax0)

newx = ini_geom.y[::-1]/1000
newy = ini_geom.x/1000

## ax0
# Plot Hb
ax0.pcolormesh(     newx, newy, ini_geom.Hb.T,  cmap='BrBG', norm=norm_bed)
CD = ax0.contour(   newx, newy, ini_geom.Hb.T,  [-600,-400,-200,0,200], colors='k')
ax0.clabel(CD, [ -600, -400, -200, 0, 200], inline=5,inline_spacing=10, fontsize=8, manual = [(0, -100), (0, -200), (0,-300), (-35,-200), (-35,-300)])

h0, l = CD.legend_elements()
# Add bathymetry 'low'
ax0.scatter([0],[0],color='k')

## ax1
# Plot ice shelf surface height on grounded ice
ax1.pcolormesh( newx, newy,  ini_geom.Hs.T, cmap='Greys_r', norm = mpl.colors.Normalize(vmin=0, vmax=1500))

# Plot mask GL and ocean
ax1.pcolormesh( newx, newy,  xr.where(ini_geom.mask.T==7, 1, np.nan),      cmap='Reds',     norm = mpl.colors.Normalize(vmin=1, vmax=2))
ax1.pcolormesh( newx, newy,  xr.where(ini_geom.mask.T==1, 1, np.nan),      cmap=cmap_ocean, norm=norm_ocean)

# Plot draft and draft contours
ax1.pcolormesh( ini_geom.y[::-1]/1000, ini_geom.x/1000, xr.where(np.logical_or(ini_geom.mask.T==4,ini_geom.mask.T==8), ini_geom.Hs.T-ini_geom.Hi.T, np.nan), cmap=cmap_draft, norm = norm_draft)
CD = ax1.contour(ini_geom.y[::-1]/1000, ini_geom.x/1000, xr.where(np.logical_or(ini_geom.mask.T==4,ini_geom.mask.T==8), ini_geom.Hs.T-ini_geom.Hi.T, np.nan),  [-300, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
h1, l = CD.legend_elements()

# Set aspect
ax0.set_aspect(.5)
ax1.set_aspect(.5)

# Add labels, title, ticks
ax0.set_xlabel('Y [km]')
ax0.set_ylabel('X [km]')
ax1.set_xlabel('Y [km]')
ax0.set_title('Bathymetry')
ax1.set_title('Ice geometry')
plt.setp(ax1.get_yticklabels(), visible=False)
ax0.set_xticks([-40,0,40])
ax1.set_xticks([-40,0,40])

# Add ocean, grounded to ax1:
ax1.text(42, 300, 'ocean',color='k', weight='bold',rotation=-90)
ax1.text(42, -300, 'grounded',color='k', weight='bold',rotation=-90)
ax1.text(42, 150, 'shelf',color='k', weight='bold',rotation=-90)

ax1.legend(h1,['-300 m', '-250 m', '-225 m', '-200 m', '-175 m', '-150 m', '-125 m', '-100 m'],bbox_to_anchor=(1.3, 1),loc=2)

fig.savefig('fig02_panel_b.png', dpi=300, bbox_inches='tight')

